## Loading the Data

In [128]:
#!pip install tensorflow
from tensorflow import keras 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Use the keras module to import the necessary data 
from sklearn.model_selection import train_test_split

In [129]:
#  Preparing and cleaning the data
df = pd.read_csv("palmer_penguins.csv")
df = df.dropna()
df = df.drop('Unnamed: 0', 1)

Given a set of feature vector of penguin characteris, can we now construct a model that will be able to guess what kind of penguin (Adelie, Gentoo or Chinstrap) just from the feature vector. Let's try to test it out.

Before we do that we have to prepare our feature vector and our target vector.

## Preparing Target Vector

In [130]:
cls = {"Adelie": 0,
       "Chinstrap": 1,
       "Gentoo": 2,}
df["species_index"] = df["species"].apply(lambda x: cls[x])
y_total = df['species_index'].values

onehot_train_y = []
for y in y_total:   
    temp_vec = np.zeros((3, 1))
    temp_vec[y][0] = 1.0
    onehot_train_y.append(temp_vec)

## Preparing Feature Vector

In [131]:
df["isFemale"] = np.where(df["sex"]=="Female",1,0)
df["isTorgersen"] = np.where(df["island"]=="Torgersen",1,0)
df["isBiscoe"] = np.where(df["island"]=="Biscoe",1,0)
df["isDream"] = np.where(df["island"]=="Dream",1,0)

X =  df[[ "bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","isFemale","isTorgersen","isBiscoe","isDream"]].values 

X[0:,0] = X[0:,0]/10
X[0:,1] = X[0:,1]/10
X[0:,2] = X[0:,2]/100
X[0:,3] = X[0:,3]/1000
np.shape(X[0])

flat_train_X = []
for x in X:   
    flat_train_X.append(x.flatten().reshape(8, 1))
    
print(np.shape(flat_train_X))

(333, 8, 1)


## Train/Test Split

In [132]:
#convert qualitative data to quantitative data
# 1. convert sex to isFemale
# 2. convert island to isTorgerson, isDream, isBiscoe boolean variables
# 3. convert species 3 valued column vector where adelie, gentoo, chinstrap [1,0,0] for adelie, [0,1,0] for chinstrap etc.

X_train, X_test, y_train, y_test = train_test_split (X, y_total, test_size= 0.3, random_state =42)

## Initializing a Mutli Layer Perceptron

In [133]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.0


In [134]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(8,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(3)
])

In [139]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=25)

Epoch 1/25
8/8 [==============================] - 1s 2ms/step - loss: 0.3418 - accuracy: 0.9013
Epoch 2/25
8/8 [==============================] - 0s 2ms/step - loss: 0.3404 - accuracy: 0.8627
Epoch 3/25
8/8 [==============================] - 0s 3ms/step - loss: 0.3268 - accuracy: 0.8927
Epoch 4/25
8/8 [==============================] - 0s 3ms/step - loss: 0.3171 - accuracy: 0.9227
Epoch 5/25
8/8 [==============================] - 0s 3ms/step - loss: 0.3031 - accuracy: 0.8884
Epoch 6/25
8/8 [==============================] - 0s 3ms/step - loss: 0.2935 - accuracy: 0.9313
Epoch 7/25
8/8 [==============================] - 0s 4ms/step - loss: 0.2848 - accuracy: 0.9313
Epoch 8/25
8/8 [==============================] - 0s 3ms/step - loss: 0.2597 - accuracy: 0.9399
Epoch 9/25
8/8 [==============================] - 0s 3ms/step - loss: 0.2775 - accuracy: 0.9227
Epoch 10/25
8/8 [==============================] - 0s 3ms/step - loss: 0.2678 - accuracy: 0.9270
Epoch 11/25
8/8 [======================

In [140]:
model.evaluate(X_test, y_test, verbose=1)

4/4 [==============================] - 0s 2ms/step - loss: 0.1410 - accuracy: 0.9900


[0.14104168117046356, 0.9900000095367432]